# 請結合前面的知識與程式碼，比較不同的 optimizer 與 learning rate 組合對訓練的結果與影響
### 常見的 optimizer 包含
* SGD
* RMSprop
* AdaGrad
* Adam

In [9]:
import os
import keras
import itertools

# Disable GPU
os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [2]:
train, test = keras.datasets.cifar10.load_data()

In [3]:
## 資料前處理
def preproc_x(x, flatten=True):
    x = x / 255.
    if flatten:
        x = x.reshape((len(x), -1))
    return x

def preproc_y(y, num_classes=10):
    if y.shape[-1] == 1:
        y = keras.utils.to_categorical(y, num_classes)
    return y    

In [4]:
x_train, y_train = train
x_test, y_test = test

# Preproc the inputs
x_train = preproc_x(x_train)
x_test = preproc_x(x_test)

# Preprc the outputs
y_train = preproc_y(y_train)
y_test = preproc_y(y_test)

In [5]:
"""
建立神經網路
"""
def build_mlp(input_shape, output_units=10, num_neurons=[512, 256, 128]):
    input_layer = keras.layers.Input(input_shape)
    
    for i, n_units in enumerate(num_neurons):
        if i == 0:
            x = keras.layers.Dense(units=n_units, activation="relu", name="hidden_layer"+str(i+1))(input_layer)
        else:
            x = keras.layers.Dense(units=n_units, activation="relu", name="hidden_layer"+str(i+1))(x)
    
    out = keras.layers.Dense(units=output_units, activation="softmax", name="output")(x)
    
    model = keras.models.Model(inputs=[input_layer], outputs=[out])
    return model

In [6]:

"""
Set your required experiment parameters
"""
## 超參數設定
LEARNING_RATE = [1e-1, 1e-2, 1e-3, 1e-4, 1e-5]
EPOCHS = 50
BATCH_SIZE = 256
OPTIMIZER = [keras.optimizers.SGD, keras.optimizers.RMSprop, keras.optimizers.Adagrad, keras.optimizers.Adam]


In [ ]:

"""
建立你的訓練與實驗迴圈並蒐集資料
"""
results = {}
for lr, opti in itertools.product(LEARNING_RATE, OPTIMIZER):
    keras.backend.clear_session() # 把舊的 Graph 清掉
    print("Experiment with LR = %.6f, Optimizer = %s" % (lr, str(opti)))
    model = build_mlp(input_shape=x_train.shape[1:])
    model.summary()
    
    optimizer = opti(lr=lr)
    model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer=optimizer)

    model.fit(x_train, y_train, 
              epochs=EPOCHS, 
              batch_size=BATCH_SIZE, 
              validation_data=(x_test, y_test), 
              shuffle=True)
    
    # Collect results
    train_loss = model.history.history["loss"]
    valid_loss = model.history.history["val_loss"]
    train_acc = model.history.history["accuracy"]
    valid_acc = model.history.history["val_accuracy"]
    
    exp_name_tag = "exp-lr-%s-optimizer-%s" % (str(lr), str(opti))
    results[exp_name_tag] = {'train-loss': train_loss,
                             'valid-loss': valid_loss,
                             'train-acc': train_acc,
                             'valid-acc': valid_acc}

Experiment with LR = 0.100000, Optimizer = <class 'keras.optimizers.SGD'>
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 3072)              0         
_________________________________________________________________
hidden_layer1 (Dense)        (None, 512)               1573376   
_________________________________________________________________
hidden_layer2 (Dense)        (None, 256)               131328    
_________________________________________________________________
hidden_layer3 (Dense)        (None, 128)               32896     
_________________________________________________________________
output (Dense)               (None, 10)                1290      
Total params: 1,738,890
Trainable params: 1,738,890
Non-trainable params: 0
_________________________________________________________________
Train on 50000 samples, validate on 10000

50000/50000 [==============================] - 12s 249us/step - loss: 0.7968 - accuracy: 0.7161 - val_loss: 1.6393 - val_accuracy: 0.4934
Epoch 49/50
50000/50000 [==============================] - 14s 279us/step - loss: 0.7746 - accuracy: 0.7239 - val_loss: 1.8104 - val_accuracy: 0.4607
Epoch 50/50
50000/50000 [==============================] - 14s 277us/step - loss: 0.7611 - accuracy: 0.7276 - val_loss: 1.6255 - val_accuracy: 0.5185
Experiment with LR = 0.100000, Optimizer = <class 'keras.optimizers.RMSprop'>
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 3072)              0         
_________________________________________________________________
hidden_layer1 (Dense)        (None, 512)               1573376   
_________________________________________________________________
hidden_layer2 (Dense)        (None, 256)               131328    
_____

50000/50000 [==============================] - 19s 384us/step - loss: 2.3080 - accuracy: 0.0987 - val_loss: 2.3232 - val_accuracy: 0.10002.3080 - accura
Epoch 45/50
50000/50000 [==============================] - 17s 332us/step - loss: 2.3077 - accuracy: 0.1010 - val_loss: 2.3158 - val_accuracy: 0.1000
Epoch 46/50
50000/50000 [==============================] - 17s 343us/step - loss: 2.3082 - accuracy: 0.0983 - val_loss: 2.3084 - val_accuracy: 0.1000
Epoch 47/50
50000/50000 [==============================] - 17s 334us/step - loss: 2.3079 - accuracy: 0.0996 - val_loss: 2.3090 - val_accuracy: 0.1000
Epoch 48/50
50000/50000 [==============================] - 17s 343us/step - loss: 2.3077 - accuracy: 0.1016 - val_loss: 2.3224 - val_accuracy: 0.1000
Epoch 49/50
50000/50000 [==============================] - 18s 359us/step - loss: 2.3082 - accuracy: 0.0969 - val_loss: 2.3130 - val_accuracy: 0.1000
Epoch 50/50
50000/50000 [==============================] - 18s 351us/step - loss: 2.3080 - accura

50000/50000 [==============================] - 12s 250us/step - loss: 1.4462 - accuracy: 0.4845 - val_loss: 1.5734 - val_accuracy: 0.4338loss: 1.4
Epoch 42/50
50000/50000 [==============================] - 12s 242us/step - loss: 1.4400 - accuracy: 0.4877 - val_loss: 1.6073 - val_accuracy: 0.4304
Epoch 43/50
50000/50000 [==============================] - 12s 240us/step - loss: 1.4352 - accuracy: 0.4880 - val_loss: 1.8085 - val_accuracy: 0.3800
Epoch 44/50
50000/50000 [==============================] - 12s 242us/step - loss: 1.4361 - accuracy: 0.4874 - val_loss: 1.5826 - val_accuracy: 0.4459
Epoch 45/50
50000/50000 [==============================] - 12s 245us/step - loss: 1.4246 - accuracy: 0.4929 - val_loss: 1.5516 - val_accuracy: 0.4570
Epoch 46/50
50000/50000 [==============================] - 12s 237us/step - loss: 1.4226 - accuracy: 0.4944 - val_loss: 1.5435 - val_accuracy: 0.4471
Epoch 47/50
50000/50000 [==============================] - 12s 243us/step - loss: 1.4167 - accuracy: 0.

50000/50000 [==============================] - 14s 278us/step - loss: 2.3073 - accuracy: 0.0989 - val_loss: 2.3081 - val_accuracy: 0.1000
Epoch 39/50
50000/50000 [==============================] - 15s 304us/step - loss: 2.3067 - accuracy: 0.0982 - val_loss: 2.3061 - val_accuracy: 0.1000
Epoch 40/50
50000/50000 [==============================] - 14s 274us/step - loss: 2.3065 - accuracy: 0.1034 - val_loss: 2.3052 - val_accuracy: 0.1000
Epoch 41/50
50000/50000 [==============================] - 14s 284us/step - loss: 2.3069 - accuracy: 0.0956 - val_loss: 2.3073 - val_accuracy: 0.1000
Epoch 42/50
50000/50000 [==============================] - 15s 295us/step - loss: 2.3076 - accuracy: 0.0989 - val_loss: 2.3063 - val_accuracy: 0.1000
Epoch 43/50
50000/50000 [==============================] - 15s 304us/step - loss: 2.3075 - accuracy: 0.1011 - val_loss: 2.3060 - val_accuracy: 0.1000
Epoch 44/50
50000/50000 [==============================] - 14s 276us/step - loss: 2.3070 - accuracy: 0.0978 - va

50000/50000 [==============================] - 11s 218us/step - loss: 1.9017 - accuracy: 0.2971 - val_loss: 1.9022 - val_accuracy: 0.2850
Epoch 11/50
50000/50000 [==============================] - 12s 248us/step - loss: 1.8933 - accuracy: 0.3017 - val_loss: 1.9828 - val_accuracy: 0.2657
Epoch 12/50
50000/50000 [==============================] - 12s 248us/step - loss: 1.8810 - accuracy: 0.3030 - val_loss: 2.0816 - val_accuracy: 0.2628
Epoch 13/50
50000/50000 [==============================] - 14s 277us/step - loss: 1.8783 - accuracy: 0.3057 - val_loss: 1.9528 - val_accuracy: 0.2789
Epoch 14/50
50000/50000 [==============================] - 14s 274us/step - loss: 1.8702 - accuracy: 0.3092 - val_loss: 1.8849 - val_accuracy: 0.3031
Epoch 15/50
50000/50000 [==============================] - 14s 283us/step - loss: 1.8668 - accuracy: 0.3104 - val_loss: 1.8416 - val_accuracy: 0.3186
Epoch 16/50
50000/50000 [==============================] - 14s 272us/step - loss: 1.8597 - accuracy: 0.3167 - va

50000/50000 [==============================] - 12s 241us/step - loss: 1.5600 - accuracy: 0.4440 - val_loss: 1.6103 - val_accuracy: 0.4278
Epoch 8/50
50000/50000 [==============================] - 13s 260us/step - loss: 1.5359 - accuracy: 0.4554 - val_loss: 1.5491 - val_accuracy: 0.4517
Epoch 9/50
50000/50000 [==============================] - 12s 234us/step - loss: 1.5136 - accuracy: 0.4628 - val_loss: 1.5442 - val_accuracy: 0.4501
Epoch 10/50
50000/50000 [==============================] - 12s 233us/step - loss: 1.4896 - accuracy: 0.4707 - val_loss: 1.5290 - val_accuracy: 0.4519
Epoch 11/50
50000/50000 [==============================] - 12s 242us/step - loss: 1.4713 - accuracy: 0.4772 - val_loss: 1.5163 - val_accuracy: 0.4619
Epoch 12/50
50000/50000 [==============================] - 12s 231us/step - loss: 1.4532 - accuracy: 0.4852 - val_loss: 1.5009 - val_accuracy: 0.4673
Epoch 13/50
50000/50000 [==============================] - 12s 232us/step - loss: 1.4399 - accuracy: 0.4908 - val_

50000/50000 [==============================] - 14s 284us/step - loss: 1.7091 - accuracy: 0.3835 - val_loss: 1.6842 - val_accuracy: 0.3780
Epoch 5/50
50000/50000 [==============================] - 14s 273us/step - loss: 1.6518 - accuracy: 0.4028 - val_loss: 1.6225 - val_accuracy: 0.4180
Epoch 6/50
50000/50000 [==============================] - 14s 270us/step - loss: 1.6408 - accuracy: 0.4074 - val_loss: 1.6466 - val_accuracy: 0.4081
Epoch 7/50
50000/50000 [==============================] - 13s 270us/step - loss: 1.6246 - accuracy: 0.4178 - val_loss: 1.5969 - val_accuracy: 0.4270
Epoch 8/50
50000/50000 [==============================] - 13s 270us/step - loss: 1.6054 - accuracy: 0.4240 - val_loss: 1.5861 - val_accuracy: 0.4314
Epoch 9/50
50000/50000 [==============================] - 14s 283us/step - loss: 1.5803 - accuracy: 0.4320 - val_loss: 1.6589 - val_accuracy: 0.3978
Epoch 10/50
50000/50000 [==============================] - 14s 280us/step - loss: 1.5806 - accuracy: 0.4319 - val_los

50000/50000 [==============================] - 10s 204us/step - loss: 2.2635 - accuracy: 0.1526 - val_loss: 2.2104 - val_accuracy: 0.2080
Epoch 2/50
50000/50000 [==============================] - 11s 218us/step - loss: 2.1805 - accuracy: 0.2208 - val_loss: 2.1505 - val_accuracy: 0.2275
Epoch 3/50
50000/50000 [==============================] - 10s 208us/step - loss: 2.1278 - accuracy: 0.2419 - val_loss: 2.1039 - val_accuracy: 0.2481
Epoch 4/50
50000/50000 [==============================] - 10s 202us/step - loss: 2.0850 - accuracy: 0.2599 - val_loss: 2.0648 - val_accuracy: 0.2699
Epoch 5/50
50000/50000 [==============================] - 10s 197us/step - loss: 2.0492 - accuracy: 0.2766 - val_loss: 2.0327 - val_accuracy: 0.2848
Epoch 6/50
50000/50000 [==============================] - 10s 209us/step - loss: 2.0195 - accuracy: 0.2892 - val_loss: 2.0052 - val_accuracy: 0.3007
Epoch 7/50
50000/50000 [==============================] - 11s 210us/step - loss: 1.9943 - accuracy: 0.3016 - val_loss

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline 
    
NUM_COLORS = len(results.keys())
cm = plt.get_cmap('gist_rainbow')
color_bar = [cm(1.*i/NUM_COLORS) for i in range(NUM_COLORS)]

plt.figure(figsize=(8,6))
for i, cond in enumerate(results.keys()):
    plt.plot(range(len(results[cond]['train-loss'])),results[cond]['train-loss'], '-', label=cond, color=color_bar[i])
    plt.plot(range(len(results[cond]['valid-loss'])),results[cond]['valid-loss'], '--', label=cond, color=color_bar[i])
plt.title("Loss")
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()

plt.figure(figsize=(8,6))
for i, cond in enumerate(results.keys()):
    plt.plot(range(len(results[cond]['train-acc'])),results[cond]['train-acc'], '-', label=cond, color=color_bar[i])
    plt.plot(range(len(results[cond]['valid-acc'])),results[cond]['valid-acc'], '--', label=cond, color=color_bar[i])
plt.title("Accuracy")
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()